# Multi-Ticker RL Trading System Examples

This notebook provides examples and tutorials for using the Multi-Ticker RL Trading System. It covers basic usage, advanced features, and custom implementations.

## Setup

First, let's import the necessary modules and set up the environment.

In [ ]:
# Import standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import os
import sys
from datetime import datetime, timedelta
from pathlib import Path

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Add project root to path
project_root = Path().resolve().parent
sys.path.append(str(project_root))

# Import project modules
from src.utils.config_loader import load_config
from src.data.multiticker_data_loader import MultiTickerDataLoader
from src.features.multiticker_pipeline import MultiTickerFeaturePipeline
from src.sim.multiticker_env import MultiTickerIntradayRLEnv
from src.rl.multiticker_trainer import MultiTickerRLTrainer
from src.rl.multiticker_policy import MultiTickerPPOLSTMPolicy
from src.evaluation.multiticker_evaluator import MultiTickerEvaluator
from src.monitoring.multiticker_monitor import MultiTickerMonitor

# Set logging
import logging
logging.basicConfig(level=logging.INFO)

print("Setup completed successfully!")

## Example 1: Basic Multi-Ticker Training

In this example, we'll train a basic multi-ticker RL model using the default configuration.

In [ ]:
# Load configuration
config = load_config()

# Print configuration overview
print("Configuration Overview:")
print(f"- Tickers: {config['data']['tickers']}")
print(f"- Start Date: {config['data']['start_date']}")
print(f"- End Date: {config['data']['end_date']}")
print(f"- Reward Type: {config['environment']['reward_type']}")
print(f"- Training Timesteps: {config['training']['total_timesteps']}")

In [ ]:
# Initialize data loader
data_loader = MultiTickerDataLoader(config['data'])

# Load data
print("Loading market data...")
data = data_loader.load_data()

print(f"Data loaded successfully!")
print(f"Data shape: {data.shape}")
print(f"Date range: {data.index.min()} to {data.index.max()}")
print(f"Number of tickers: {len(data.columns.get_level_values('ticker').unique())}")

In [ ]:
# Initialize feature pipeline
feature_pipeline = MultiTickerFeaturePipeline(config['features'])

# Fit and transform features
print("Extracting features...")
features = feature_pipeline.fit_transform(data)

print(f"Features extracted successfully!")
print(f"Features shape: {features.shape}")
print(f"Number of features: {features.shape[1]}")

In [ ]:
# Initialize trainer
trainer = MultiTickerRLTrainer(config)

# Train model (reduced timesteps for example)
print("Training model...")
config['training']['total_timesteps'] = 10000  # Reduced for example
model = trainer.train()

print("Model trained successfully!")

In [ ]:
# Save model
model_path = "models/basic_multiticker_model"
trainer.save_model(model_path)

print(f"Model saved to {model_path}")

## Example 2: Model Evaluation

Now let's evaluate the trained model on test data.

In [ ]:
# Load model
model = MultiTickerPPOLSTMPolicy.load(model_path)

# Initialize evaluator
evaluator = MultiTickerEvaluator(config['evaluation'])

# Split data for training and testing
split_date = data.index.max() - timedelta(days=30)
train_data = data[data.index <= split_date]
test_data = data[data.index > split_date]

print(f"Training data: {train_data.index.min()} to {train_data.index.max()}")
print(f"Test data: {test_data.index.min()} to {test_data.index.max()}")

In [ ]:
# Evaluate model
print("Evaluating model...")
results = evaluator.evaluate_model(model, test_data)

# Print key metrics
print("\nEvaluation Results:")
print(f"Sharpe Ratio: {results['sharpe_ratio']:.2f}")
print(f"Total Return: {results['total_return']:.2%}")
print(f"Annualized Return: {results['annualized_return']:.2%}")
print(f"Max Drawdown: {results['max_drawdown']:.2%}")
print(f"Win Rate: {results['win_rate']:.2%}")
print(f"Profit Factor: {results['profit_factor']:.2f}")

In [ ]:
# Plot equity curve
plt.figure(figsize=(12, 6))
equity_curve = pd.DataFrame(results['equity_curve'])
equity_curve.set_index('timestamp', inplace=True)
plt.plot(equity_curve.index, equity_curve['equity'], label='Portfolio Value')
plt.title('Portfolio Equity Curve')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot drawdown
plt.figure(figsize=(12, 6))
equity_series = equity_curve['equity']
cummax = equity_series.cummax()
drawdown = (equity_series - cummax) / cummax
plt.fill_between(drawdown.index, drawdown, 0, alpha=0.3, color='red')
plt.plot(drawdown.index, drawdown, color='red')
plt.title('Portfolio Drawdown')
plt.xlabel('Date')
plt.ylabel('Drawdown (%)')
plt.grid(True)
plt.show()

## Example 3: Walk-Forward Optimization

In this example, we'll implement walk-forward optimization with Leave-One-Ticker-Out cross-validation.

In [ ]:
# Configure walk-forward optimization
wfo_config = config.copy()
wfo_config['walkforward'] = {
    'enabled': True,
    'n_folds': 3,  # Reduced for example
    'embargo_days': 5,
    'test_size': 0.2,
    'regime_aware': True
}

# Initialize trainer with WFO
wfo_trainer = MultiTickerRLTrainer(wfo_config)

# Run walk-forward optimization
print("Running walk-forward optimization...")
wfo_results = wfo_trainer.walk_forward_training()

print("Walk-forward optimization completed!")

In [ ]:
# Print WFO results
print("\nWalk-Forward Optimization Results:")
print(f"Average Sharpe Ratio: {wfo_results['avg_sharpe_ratio']:.2f}")
print(f"Average Total Return: {wfo_results['avg_total_return']:.2%}")
print(f"Average Max Drawdown: {wfo_results['avg_max_drawdown']:.2%}")
print(f"Fold Results: {wfo_results['fold_results']}")

In [ ]:
# Plot fold performance
plt.figure(figsize=(12, 6))
fold_results = wfo_results['fold_results']
fold_names = list(fold_results.keys())
sharpe_ratios = [fold_results[fold]['sharpe_ratio'] for fold in fold_names]
total_returns = [fold_results[fold]['total_return'] for fold in fold_names]

x = np.arange(len(fold_names))
width = 0.35

fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot Sharpe ratios
ax1.bar(x - width/2, sharpe_ratios, width, label='Sharpe Ratio', color='skyblue')
ax1.set_xlabel('Fold')
ax1.set_ylabel('Sharpe Ratio', color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')
ax1.set_xticks(x)
ax1.set_xticklabels(fold_names)

# Plot total returns on secondary axis
ax2 = ax1.twinx()
ax2.bar(x + width/2, total_returns, width, label='Total Return', color='lightgreen')
ax2.set_ylabel('Total Return', color='lightgreen')
ax2.tick_params(axis='y', labelcolor='lightgreen')

plt.title('Walk-Forward Optimization Fold Performance')
fig.tight_layout()
plt.show()

## Example 4: Hyperparameter Optimization

In this example, we'll use Optuna to optimize hyperparameters for the multi-ticker RL model.

In [ ]:
# Configure hyperparameter optimization
hpo_config = config.copy()
hpo_config['hpo'] = {
    'enabled': True,
    'n_trials': 10,  # Reduced for example
    'direction': 'maximize',
    'metric': 'sharpe_ratio',
    'pruner': 'median',
    'sampler': 'tpe',
    'search_space': {
        'learning_rate': {'type': 'float', 'low': 1e-5, 'high': 1e-3, 'log': True},
        'batch_size': {'type': 'categorical', 'choices': [32, 64, 128]},
        'n_steps': {'type': 'categorical', 'choices': [1024, 2048, 4096]},
        'gamma': {'type': 'float', 'low': 0.9, 'high': 0.999},
        'gae_lambda': {'type': 'float', 'low': 0.8, 'high': 0.99},
        'clip_range': {'type': 'float', 'low': 0.1, 'high': 0.4},
        'ent_coef': {'type': 'float', 'low': 1e-4, 'high': 0.1, 'log': True},
        'vf_coef': {'type': 'float', 'low': 0.1, 'high': 1.0}
    }
}

# Initialize trainer with HPO
hpo_trainer = MultiTickerRLTrainer(hpo_config)

# Run hyperparameter optimization
print("Running hyperparameter optimization...")
hpo_results = hpo_trainer.optimize_hyperparameters()

print("Hyperparameter optimization completed!")

In [ ]:
# Print HPO results
print("\nHyperparameter Optimization Results:")
print("Best parameters:")
for param, value in hpo_results['best_params'].items():
    print(f"  {param}: {value}")

print(f"\nBest Sharpe Ratio: {hpo_results['best_value']:.2f}")
print(f"Number of trials: {len(hpo_results['history'])}")

In [ ]:
# Plot optimization history
plt.figure(figsize=(12, 6))
history = hpo_results['history']
trials = list(range(1, len(history) + 1))
values = [trial['value'] for trial in history]

plt.plot(trials, values, 'o-')
plt.axhline(y=hpo_results['best_value'], color='r', linestyle='--', label='Best Value')
plt.title('Hyperparameter Optimization History')
plt.xlabel('Trial')
plt.ylabel('Sharpe Ratio')
plt.legend()
plt.grid(True)
plt.show()

## Example 5: Custom Reward Function

In this example, we'll implement a custom reward function that emphasizes diversification and risk management.

In [ ]:
# Define custom reward calculator
from src.sim.multiticker_env import MultiTickerRewardCalculator

class CustomRewardCalculator(MultiTickerRewardCalculator):
    def __init__(self, config):
        super().__init__(config)
        
    def calculate_reward(self, portfolio_state, action, next_portfolio_state):
        # Calculate base reward
        base_reward = super().calculate_reward(portfolio_state, action, next_portfolio_state)
        
        # Add custom reward for diversification
        diversification_reward = self._calculate_diversification_reward(portfolio_state)
        
        # Add custom penalty for concentration
        concentration_penalty = self._calculate_concentration_penalty(portfolio_state)
        
        # Add custom reward for risk-adjusted returns
        risk_adjusted_reward = self._calculate_risk_adjusted_reward(portfolio_state, next_portfolio_state)
        
        # Combine rewards
        total_reward = base_reward + diversification_reward - concentration_penalty + risk_adjusted_reward
        
        return total_reward
    
    def _calculate_diversification_reward(self, portfolio_state):
        positions = portfolio_state['positions']
        if len(positions) <= 1:
            return 0.0
        
        # Calculate Herfindahl-Hirschman Index (HHI)
        total_value = sum(abs(pos) for pos in positions.values())
        if total_value == 0:
            return 0.0
        
        hhi = sum((abs(pos) / total_value) ** 2 for pos in positions.values())
        
        # Reward for lower HHI (more diversified)
        diversification_reward = (1 - hhi) * 0.1
        
        return diversification_reward
    
    def _calculate_concentration_penalty(self, portfolio_state):
        positions = portfolio_state['positions']
        if len(positions) == 0:
            return 0.0
        
        # Calculate concentration
        total_value = sum(abs(pos) for pos in positions.values())
        if total_value == 0:
            return 0.0
        
        max_concentration = max(abs(pos) / total_value for pos in positions.values())
        
        # Penalty for high concentration
        if max_concentration > 0.4:
            concentration_penalty = (max_concentration - 0.4) * 0.2
        else:
            concentration_penalty = 0.0
        
        return concentration_penalty
    
    def _calculate_risk_adjusted_reward(self, portfolio_state, next_portfolio_state):
        # Calculate portfolio returns
        current_value = portfolio_state['portfolio_value']
        next_value = next_portfolio_state['portfolio_value']
        
        if current_value == 0:
            return 0.0
        
        portfolio_return = (next_value - current_value) / current_value
        
        # Calculate portfolio volatility (simplified)
        positions = portfolio_state['positions']
        if len(positions) == 0:
            return 0.0
        
        # Use position sizes as proxy for risk
        position_sizes = [abs(pos) for pos in positions.values()]
        portfolio_volatility = np.std(position_sizes) / np.mean(position_sizes) if np.mean(position_sizes) > 0 else 0.0
        
        # Calculate risk-adjusted return (simplified Sharpe ratio)
        if portfolio_volatility > 0:
            risk_adjusted_return = portfolio_return / portfolio_volatility
        else:
            risk_adjusted_return = 0.0
        
        # Scale reward
        risk_adjusted_reward = risk_adjusted_return * 0.05
        
        return risk_adjusted_reward

In [ ]:
# Configure custom reward training
custom_reward_config = config.copy()
custom_reward_config['environment']['reward_calculator'] = CustomRewardCalculator
custom_reward_config['training']['total_timesteps'] = 10000  # Reduced for example

# Initialize trainer with custom reward
custom_trainer = MultiTickerRLTrainer(custom_reward_config)

# Train model with custom reward
print("Training model with custom reward function...")
custom_model = custom_trainer.train()

print("Model with custom reward function trained successfully!")

In [ ]:
# Evaluate custom reward model
print("Evaluating custom reward model...")
custom_results = evaluator.evaluate_model(custom_model, test_data)

# Print key metrics
print("\nCustom Reward Model Evaluation Results:")
print(f"Sharpe Ratio: {custom_results['sharpe_ratio']:.2f}")
print(f"Total Return: {custom_results['total_return']:.2%}")
print(f"Annualized Return: {custom_results['annualized_return']:.2%}")
print(f"Max Drawdown: {custom_results['max_drawdown']:.2%}")
print(f"Win Rate: {custom_results['win_rate']:.2%}")
print(f"Profit Factor: {custom_results['profit_factor']:.2f}")

In [ ]:
# Compare with baseline model
plt.figure(figsize=(12, 6))

# Plot equity curves
baseline_equity = pd.DataFrame(results['equity_curve'])
baseline_equity.set_index('timestamp', inplace=True)
plt.plot(baseline_equity.index, baseline_equity['equity'], label='Baseline Model')

custom_equity = pd.DataFrame(custom_results['equity_curve'])
custom_equity.set_index('timestamp', inplace=True)
plt.plot(custom_equity.index, custom_equity['equity'], label='Custom Reward Model')

plt.title('Model Comparison: Equity Curves')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid(True)
plt.show()

## Example 6: Dynamic Universe Selection

In this example, we'll implement a dynamic universe selection strategy that adapts to changing market conditions.

In [ ]:
# Define custom universe selector
from src.data.multiticker_data_loader import MultiTickerDataLoader

class DynamicUniverseSelector:
    def __init__(self, config):
        self.config = config
        self.max_tickers = config.get('max_tickers', 10)
        self.min_tickers = config.get('min_tickers', 3)
        self.rebalance_freq = config.get('rebalance_freq', '1M')
        self.selection_metrics = config.get('selection_metrics', ['liquidity', 'volatility', 'trend_strength'])
        
    def select_universe(self, data, current_universe=None, date=None):
        """
        Select universe of tickers based on custom criteria.
        """
        # Get all available tickers
        all_tickers = data.columns.get_level_values('ticker').unique()
        
        # Calculate selection metrics for each ticker
        ticker_metrics = {}
        
        for ticker in all_tickers:
            if (ticker, 'close') in data.columns:
                prices = data[(ticker, 'close')]
                volume = data[(ticker, 'volume')] if (ticker, 'volume') in data.columns else None
                
                # Calculate selection metrics
                metrics = self._calculate_selection_metrics(prices, volume)
                ticker_metrics[ticker] = metrics
        
        # Score and rank tickers
        ticker_scores = {}
        for ticker, metrics in ticker_metrics.items():
            score = self._calculate_ticker_score(metrics)
            ticker_scores[ticker] = score
        
        # Sort tickers by score
        sorted_tickers = sorted(ticker_scores.items(), key=lambda x: x[1], reverse=True)
        
        # Select top tickers
        selected_tickers = [ticker for ticker, _ in sorted_tickers[:self.max_tickers]]
        
        # Ensure minimum number of tickers
        if len(selected_tickers) < self.min_tickers:
            selected_tickers = [ticker for ticker, _ in sorted_tickers[:self.min_tickers]]
        
        return selected_tickers
    
    def _calculate_selection_metrics(self, prices, volume=None):
        metrics = {}
        
        # Liquidity metric
        if volume is not None:
            metrics['liquidity'] = volume.mean()
        else:
            metrics['liquidity'] = 1.0
        
        # Volatility metric
        returns = prices.pct_change().dropna()
        metrics['volatility'] = returns.std()
        
        # Trend strength metric
        metrics['trend_strength'] = self._calculate_trend_strength(prices)
        
        # Price momentum
        metrics['momentum'] = (prices.iloc[-1] / prices.iloc[-21]) - 1 if len(prices) >= 21 else 0.0
        
        return metrics
    
    def _calculate_trend_strength(self, prices):
        # Calculate trend strength using linear regression
        x = np.arange(len(prices))
        y = prices.values
        
        # Remove NaN values
        mask = ~np.isnan(y)
        x = x[mask]
        y = y[mask]
        
        if len(x) < 2:
            return 0.0
        
        # Calculate linear regression
        slope, _ = np.polyfit(x, y, 1)
        
        # Normalize slope by price level
        trend_strength = slope / np.mean(y) if np.mean(y) != 0 else 0.0
        
        return trend_strength
    
    def _calculate_ticker_score(self, metrics):
        # Calculate composite score from metrics
        liquidity_score = min(metrics['liquidity'] / 1e6, 1.0)
        volatility_score = min(metrics['volatility'] / 0.02, 1.0)
        trend_score = min(abs(metrics['trend_strength']) * 100, 1.0)
        momentum_score = min(abs(metrics['momentum']) * 10, 1.0)
        
        # Calculate weighted score
        score = (
            0.2 * liquidity_score +
            0.3 * volatility_score +
            0.25 * trend_score +
            0.25 * momentum_score
        )
        
        return score

In [ ]:
# Configure dynamic universe selection
dynamic_universe_config = config.copy()
dynamic_universe_config['multiticker'] = {
    'universe': {
        'selection_method': 'dynamic',
        'max_tickers': 5,
        'min_tickers': 3,
        'rebalance_freq': '1M',
        'selection_metrics': ['liquidity', 'volatility', 'trend_strength'],
        'selector_class': DynamicUniverseSelector
    }
}

# Initialize data loader with dynamic universe
dynamic_data_loader = MultiTickerDataLoader(dynamic_universe_config['data'])
dynamic_data_loader.universe_selector = DynamicUniverseSelector(dynamic_universe_config['multiticker']['universe'])

# Load data with dynamic universe
print("Loading data with dynamic universe selection...")
dynamic_data = dynamic_data_loader.load_data()

print(f"Dynamic universe data loaded successfully!")
print(f"Data shape: {dynamic_data.shape}")
print(f"Number of tickers: {len(dynamic_data.columns.get_level_values('ticker').unique())}")

In [ ]:
# Simulate dynamic universe selection over time
print("Simulating dynamic universe selection over time...")

# Get monthly dates
monthly_dates = pd.date_range(start=data.index.min(), end=data.index.max(), freq='MS')

universe_history = []

for date in monthly_dates:
    # Get data up to current date
    data_up_to_date = data[data.index <= date]
    
    if len(data_up_to_date) > 21:  # Ensure we have enough data
        # Select universe
        universe = dynamic_data_loader.universe_selector.select_universe(data_up_to_date, date=date)
        
        universe_history.append({
            'date': date,
            'universe': universe
        })

# Print universe selection history
print("\nDynamic Universe Selection History:")
for entry in universe_history[-5:]:  # Show last 5 months
    print(f"{entry['date'].strftime('%Y-%m-%d')}: {entry['universe']}")

## Example 7: Performance Benchmarking

In this example, we'll benchmark the RL model against various baseline strategies.

In [ ]:
# Import benchmarking framework
from tests.performance_benchmarking import PerformanceBenchmarkingFramework, BenchmarkStrategy

# Initialize benchmarking framework
benchmark_config = config.copy()
benchmark_config['benchmarking'] = {
    'output_dir': '/tmp/benchmarking_results',
    'significance_level': 0.05
}

benchmark_framework = PerformanceBenchmarkingFramework(benchmark_config)

# Add default benchmark strategies
benchmark_strategies = benchmark_framework.add_default_benchmark_strategies()

print(f"Added {len(benchmark_strategies)} benchmark strategies:")
for strategy in benchmark_strategies:
    print(f"  - {strategy.name}: {strategy.description}")

In [ ]:
# Run benchmarking analysis
print("Running benchmarking analysis...")
benchmark_results = benchmark_framework.run_benchmarks(test_data, model, parallel=False)

print("Benchmarking analysis completed!")

In [ ]:
# Print benchmarking results
print("\nBenchmarking Results:")
print("\nMetrics Comparison:")
for metric, strategies in benchmark_results['comparison']['metrics_comparison'].items():
    print(f"\n{metric.replace('_', ' ').title()}:")
    for strategy, value in strategies.items():
        print(f"  {strategy}: {value:.4f}")

In [ ]:
# Print rankings
print("\nRankings:")
for metric, ranking in benchmark_results['comparison']['ranking'].items():
    print(f"\n{metric.replace('_', ' ').title()}:")
    for i, (strategy, value) in enumerate(ranking[:3]):  # Top 3
        print(f"  {i+1}. {strategy}: {value:.4f}")

In [ ]:
# Print statistical tests
print("\nStatistical Tests:")
for strategy_name, tests in benchmark_results['statistical_tests']['t_tests'].items():
    significant = "(significant)" if tests['significant'] else "(not significant)"
    print(f"\n{strategy_name}:")
    print(f"  T-test p-value: {tests['p_value']:.4f} {significant}")
    
    if strategy_name in benchmark_results['statistical_tests']['bootstrap']:
        bootstrap_test = benchmark_results['statistical_tests']['bootstrap'][strategy_name]
        significant = "(significant)" if bootstrap_test['significant'] else "(not significant)"
        print(f"  Bootstrap p-value: {bootstrap_test['p_value']:.4f} {significant}")

In [ ]:
# Plot benchmarking results
plt.figure(figsize=(14, 8))

# Get key metrics
key_metrics = ['sharpe_ratio', 'total_return', 'max_drawdown']

for i, metric in enumerate(key_metrics):
    plt.subplot(2, 2, i+1)
    
    if metric in benchmark_results['comparison']['metrics_comparison']:
        strategies = list(benchmark_results['comparison']['metrics_comparison'][metric].keys())
        values = list(benchmark_results['comparison']['metrics_comparison'][metric].values())
        
        bars = plt.bar(strategies, values)
        
        # Highlight RL model
        if 'RL Model' in strategies:
            rl_index = strategies.index('RL Model')
            bars[rl_index].set_color('red')
        
        plt.title(metric.replace('_', ' ').title())
        plt.xticks(rotation=45)
        plt.grid(True, axis='y')

plt.tight_layout()
plt.show()

## Example 8: Real-time Monitoring

In this example, we'll set up real-time monitoring for the trading system.

In [ ]:
# Configure monitoring
monitoring_config = config.copy()
monitoring_config['monitoring'] = {
    'enabled': True,
    'output_dir': '/tmp/monitoring_results',
    'alert_thresholds': {
        'drawdown': 0.15,
        'sharpe_ratio': 0.5,
        'win_rate': 0.45,
        'var_95': 0.05
    },
    'dashboard': {
        'enabled': True,
        'port': 8080,
        'update_interval': 60
    }
}

# Initialize monitor
monitor = MultiTickerMonitor(monitoring_config['monitoring'])

print("Monitoring initialized successfully!")

In [ ]:
# Simulate real-time monitoring
print("Simulating real-time monitoring...")

# Get sample data for monitoring
sample_data = test_data.head(100)  # Use first 100 data points for simulation

# Initialize monitoring state
monitor.initialize_monitoring()

# Simulate trading and monitoring
for i, (timestamp, row) in enumerate(sample_data.iterrows()):
    # Simulate trading decision (simplified)
    if i % 5 == 0:  # Make a trade every 5 steps
        # Random trade for simulation
        ticker = np.random.choice(sample_data.columns.get_level_values('ticker').unique())
        action = np.random.choice(['buy', 'sell'])
        size = np.random.uniform(0.1, 0.3)
        
        # Record trade
        monitor.record_trade({
            'timestamp': timestamp,
            'ticker': ticker,
            'action': action,
            'size': size,
            'price': row[(ticker, 'close')]
        })
    
    # Update portfolio state
    portfolio_state = {
        'timestamp': timestamp,
        'portfolio_value': 100000 * (1 + 0.0001 * i),  # Simulated growth
        'positions': {
            'AAPL': 100,
            'MSFT': 50,
            'GOOGL': 30
        },
        'cash': 50000
    }
    
    # Update monitoring
    monitor.update_portfolio_state(portfolio_state)
    
    # Check for alerts
    alerts = monitor.check_alerts()
    
    if alerts:
        print(f"\nAlerts at {timestamp}:")
        for alert in alerts:
            print(f"  - {alert['type']}: {alert['message']}")

print("\nReal-time monitoring simulation completed!")

In [ ]:
# Get monitoring summary
summary = monitor.get_summary()

print("\nMonitoring Summary:")
print(f"Current Portfolio Value: ${summary['current_portfolio_value']:,.2f}")
print(f"Total Return: {summary['total_return']:.2%}")
print(f"Current Drawdown: {summary['current_drawdown']:.2%}")
print(f"Sharpe Ratio: {summary['sharpe_ratio']:.2f}")
print(f"Win Rate: {summary['win_rate']:.2%}")
print(f"Total Trades: {summary['total_trades']}")
print(f"Active Alerts: {len(summary['active_alerts'])}")

## Conclusion

This notebook has provided comprehensive examples and tutorials for using the Multi-Ticker RL Trading System. We've covered:

1. **Basic Multi-Ticker Training**: Training a simple multi-ticker RL model
2. **Model Evaluation**: Evaluating model performance on test data
3. **Walk-Forward Optimization**: Implementing robust validation with LOT-O CV
4. **Hyperparameter Optimization**: Using Optuna for automated hyperparameter tuning
5. **Custom Reward Function**: Implementing custom reward functions with diversification and risk management
6. **Dynamic Universe Selection**: Adapting to changing market conditions
7. **Performance Benchmarking**: Comparing against baseline strategies
8. **Real-time Monitoring**: Setting up monitoring and alerting

These examples demonstrate the flexibility and power of the Multi-Ticker RL Trading System. You can extend and customize these examples to suit your specific trading requirements and market conditions.